In [ ]:
# Packages to be imported

from ipynb.fs.full.Functions import *

In [ ]:
# Parameter Initialization
time_step =[]
Mbits_transmitted = []

# Data for training.
with open('./Traffic_Train_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans = np.array(Mbits_transmitted)
time = np.array(time_step)

In [ ]:
comms_round = 100
Clients = create_clients(series_trans,35, 'clients')

In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
        
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(Clients)
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        dpsgd = True
        learning_rate = 3e-4
        noise_multiplier= 0.9
        l2_norm_clip= 1.0
        microbatches= 1  # must evenly divide batch_size
        window_size = 20
        batch_size= 16
        epochs= 1
        dataset = windowed_dataset(np.array(Clients[client]), window_size, batch_size, len(Clients[client]))        
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        optimizer = DPKerasSGDOptimizer(l2_norm_clip=l2_norm_clip,
                                    noise_multiplier=noise_multiplier,
                                    num_microbatches=microbatches,
                                    learning_rate=learning_rate)
        #compiling the model with a mean squared error loss and a stochastic gradient descent optimizer
        local_model.compile(loss="mse", optimizer=optimizer, metrics=["mae"])
                
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(dataset, epochs=1, batch_size=batch_size)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(Clients, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

In [ ]:
# For testing the performance of the model.
with open('./Traffic_Test_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans_test = np.array(Mbits_transmitted)

# The average MSE on 9 datasets
print(get_average_mse(series_trans_test, global_model, window_size, 10))